In [1]:
import pandas as pd
from pathlib import Path

# ─── CONFIG ────────────────────────────────────────────────────────────────────
BASELINE_DIR       = Path(r"Z:\Thesis\Data\ML_Data\AP_ML_training_data")
SURFACE_PATTERN    = "*surface_monthly_stats_merged.parquet"
PRESSURE_PATTERN   = "*pressure_monthly_stats_merged.parquet"
SURFACE_PRED_PATH  = Path(r"Z:\Thesis\Data\test\DustCast\SFC\DC_v0063\sfc_row_level_predictions_2023_v0063_H3res4.csv")
# path for merra-2 2023: "Z:\Thesis\Data\test\DustCast\SFC\DC_v0063\sfc_row_level_predictions_2023_v0063_H3res4.csv"
# path for DustCast predictions: "Z:\Thesis\Data\test\DustCast\SFC\DC_v0063\sfc_monthly_ensemble_predictions_2023_v0063_H3res4.csv"
PRESSURE_PRED_PATH = Path(r"Z:\Thesis\Data\test\DustCast\UA\DC_v0063\ua_level_averages_2023_v0063_H3res4_with_MSE.csv")
# path for merra-2 2023: "Z:\Thesis\Data\test\DustCast\UA\DC_v0063\ua_level_averages_2023_v0063_H3res4_with_MSE.csv"
# path for DustCast predictions: "Z:\Thesis\Data\test\DustCast\UA\DC_v0063\ua_monthly_ensemble_predictions_2023_averaged_across_levels_v0063_H3res4.csv"
# ─── OUTPUT FOLDERS ─────────────────────────────────────────────────────────────
OUTPUT_UA_DIR  = Path(r"Z:\Thesis\Data\test\DustCast\UA\DC_v0063")
OUTPUT_SFC_DIR = Path(r"Z:\Thesis\Data\test\DustCast\SFC\DC_v0063")

# ─── STEP 1: load baseline parquet files (with filepath logging) ───────────────
def load_baseline():
    surf_dfs, pres_dfs = [], []
    for year_dir in sorted(BASELINE_DIR.iterdir()):
        if not year_dir.is_dir(): 
            continue

        # surface files
        for p in year_dir.glob(SURFACE_PATTERN):
            print(f"Loading surface file: {p}")
            df = pd.read_parquet(p, columns=["h3_res_4", "month", "DUSMASS_mean"])
            surf_dfs.append(df)

        # pressure files
        for p in year_dir.glob(PRESSURE_PATTERN):
            print(f"Loading pressure file: {p}")
            df = pd.read_parquet(p, columns=["h3_res_4", "month", "DUCMASS_mean"])
            pres_dfs.append(df)

    return pd.concat(surf_dfs, ignore_index=True), pd.concat(pres_dfs, ignore_index=True)

# ─── STEP 2: compute climatology ────────────────────────────────────────────────
def compute_climatology(df, value_col):
    return (
        df
        .groupby(["h3_res_4", "month"], as_index=False)[value_col]
        .mean()
        .rename(columns={value_col: "climatology"})
    )

# ─── STEP 3: load 2023 predictions ─────────────────────────────────────────────
def load_predictions(path, pred_col_name):
    df = pd.read_csv(path)
    df["month"] = pd.to_datetime(df["Month"], format="%Y-%m").dt.month
    return df.rename(columns={pred_col_name: "prediction"})

# ─── STEP 4: merge + compute anomaly + pct diff ─────────────────────────────────
def compute_anomaly(pred_df, clim_df):
    merged = pred_df.merge(
        clim_df,
        on=["h3_res_4", "month"],
        how="left",
        validate="many_to_one"
    )
    merged["anomaly"] = merged["prediction"] - merged["climatology"]
    merged["percent_diff"] = ((merged["anomaly"] / merged["climatology"]) * 100).round(2)
    return merged

# ─── MAIN ───────────────────────────────────────────────────────────────────────
def main():
    # ensure output dirs exist
    OUTPUT_UA_DIR.mkdir(parents=True, exist_ok=True)
    OUTPUT_SFC_DIR.mkdir(parents=True, exist_ok=True)

    print("🔄 Loading baseline data…")
    base_surf, base_pres = load_baseline()

    print("📊 Computing climatologies…")
    clim_surf = compute_climatology(base_surf, "DUSMASS_mean")
    clim_pres = compute_climatology(base_pres, "DUCMASS_mean")

    print("📥 Loading 2023 predictions…")
    pred_surf = load_predictions(SURFACE_PRED_PATH, "DUSMASS_mean") # DustCast: "ensemble_predictions"
    pred_pres = load_predictions(PRESSURE_PRED_PATH, "DUCMASS_mean") # DustCast: "ensemble_prediction_across_levels"

    print("⚖️  Computing anomalies and percent differences…")
    anom_surf = compute_anomaly(pred_surf, clim_surf)
    anom_pres = compute_anomaly(pred_pres, clim_pres)

    print("💾 Saving results…")
    out_sfc = OUTPUT_SFC_DIR / "MERRA_2_sfc_2023_anomaly.csv" # Drop MERRA_2 prefix
    out_ua  = OUTPUT_UA_DIR  / "MERRA_2_ua_2023_anomaly.csv" # Drop MERRA_2 prefix
    anom_surf.to_csv(out_sfc, index=False)
    anom_pres.to_csv(out_ua,   index=False)

    print("✅ Done. Files written to:")
    print("   •", out_sfc)
    print("   •", out_ua)

if __name__ == "__main__":
    main()


🔄 Loading baseline data…
Loading surface file: Z:\Thesis\Data\ML_Data\AP_ML_training_data\1980\Bahrain_1980_surface_monthly_stats_merged.parquet
Loading surface file: Z:\Thesis\Data\ML_Data\AP_ML_training_data\1980\Kuwait_1980_surface_monthly_stats_merged.parquet
Loading surface file: Z:\Thesis\Data\ML_Data\AP_ML_training_data\1980\Oman_1980_surface_monthly_stats_merged.parquet
Loading surface file: Z:\Thesis\Data\ML_Data\AP_ML_training_data\1980\Qatar_1980_surface_monthly_stats_merged.parquet
Loading surface file: Z:\Thesis\Data\ML_Data\AP_ML_training_data\1980\Saudi_Arabia_1980_surface_monthly_stats_merged.parquet
Loading surface file: Z:\Thesis\Data\ML_Data\AP_ML_training_data\1980\United_Arab_Emirates_1980_surface_monthly_stats_merged.parquet
Loading surface file: Z:\Thesis\Data\ML_Data\AP_ML_training_data\1980\Yemen_1980_surface_monthly_stats_merged.parquet
Loading pressure file: Z:\Thesis\Data\ML_Data\AP_ML_training_data\1980\Bahrain_1980_pressure_monthly_stats_merged.parquet
Loa

In [3]:
import os
print("CWD is:", os.getcwd())


CWD is: C:\Users\Charl\OneDrive\Documents\PSU\Thesis\Thesis V1\DustCast\Performance_eval
